In [1]:
import sumolib, math
import pandas as pd
import numpy as np
import os, glob
from datetime import datetime

### intersections
intersections_path = os.path.join('daily/S_INT_CONFIG.csv')
intersections = pd.read_csv(intersections_path)
intersections_original = intersections.copy()

# demo names
demo_names = \
["한솔삼거리(한솔마을3거리)",
"느티마을4단지",
"한솔6단지(정자동사무소)",
"신기4거리",
"느티마을4거리",
"느티마을 안촌유치원",
"상록마을 321동",
"상록4거리(상록마을입구4거리)",
"상록마을단일로",
"상록마을302동 단일로"]
intersections = intersections[intersections['INT_NAME'].isin(demo_names)]
demo_node_ids = ["n0", "n1", "n2", "n3", "n4", "n5", "n6", "n7", "n8", "n9"]
demo_int_nos = list(intersections['INT_NO'])

# SUMO node_id 정보 추가
name2node_id = dict(zip(demo_names, demo_node_ids))
intersections['node_ids'] = intersections['INT_NAME'].map(name2node_id)

# SUMO X, Y 좌표 정보 추가
origin_name = '상록4거리(상록마을입구4거리)'
ref_lng = intersections[intersections['INT_NAME'] == origin_name]['INT_LNG'].values[0]
ref_lat = intersections[intersections['INT_NAME'] == origin_name]['INT_LAT'].values[0]
intersections['X'] = (intersections['INT_LNG'] - ref_lng) * 111000 * math.cos(math.radians(ref_lat))
intersections['Y'] = (intersections['INT_LAT'] - ref_lat) * 111000
intersections.reset_index(drop=True, inplace=True)

# n=ways 정보 추가 where "n지교차로"
name2way = \
{"한솔삼거리(한솔마을3거리)":3,
"느티마을4단지":3,
"한솔6단지(정자동사무소)":4,
"신기4거리":4,
"느티마을4거리":4,
"느티마을 안촌유치원":2,
"상록마을 321동":2,
"상록4거리(상록마을입구4거리)":4,
"상록마을단일로":2,
"상록마을302동 단일로":2}
intersections['ways'] = intersections['INT_NAME'].map(name2way)

### phase_config
phase_config_path = os.path.join('daily/S_INT_PHASE_CONFIG.csv')
phase_config = pd.read_csv(phase_config_path)
phase_config_original = phase_config.copy()
phase_config = phase_config[phase_config['INT_NO'].isin(demo_int_nos)]

# INT_NO, ways 가져옴
phase_config = pd.merge(phase_config, intersections[['INT_NO', 'ways']], on='INT_NO', how='left')
cols = phase_config.columns.tolist()
cols.remove('ways')
cols.insert(1, 'ways')
phase_config = phase_config[cols]

# 링번호 변경, 주현시여부, 황색시간, 최소녹색시간, 최대녹색시간 임의 지정
phase_config['INT_RING'] = phase_config['INT_RING'].replace({0: 'A', 1: 'B'})
phase_config['INT_MAIN_PHASE'] = np.where(phase_config['INT_PHASE_NO'] == 1, 20, 10)
null_condition = phase_config['INT_FLOW_NO'] == 17
phase_config['INT_YELLOW'] = np.where(null_condition, 0, 4)
np.random.seed(0)
phase_config['INT_MIN_SPLIT'] = np.where(~null_condition, np.random.randint(12, 41, size=len(phase_config)), phase_config['INT_MIN_SPLIT'])
phase_config['INT_MAX_SPLIT'] = np.where(~null_condition, phase_config['INT_MIN_SPLIT'] + np.random.randint(5, 20, size=len(phase_config)) ,phase_config['INT_MAX_SPLIT'])
phase_config.sort_values(by=['INT_NO', 'INT_PHASE_NO','INT_RING'], inplace=True)

In [2]:
# 1. 교차로정보 (intersections, 테이블명 : S_INT_CONFIG)
intersections_original[:10]

,INT_NO,INT_NAME,SA_NO,INT_LAT,INT_LNG,IS_SMART
0,1,도촌4거리,1,37.41405,127.14385,1.0
1,2,동원SK삼거리(아시아 주유소),1,37.41282,127.14382,NaN
2,3,상탑4거리,0,37.40809,127.14296,NaN
3,4,돌마4거리(상탑3거리),0,37.40683,127.14201,1.0
4,5,상탑초3거리,0,37.40561,127.14473,NaN
5,6,야탑3치안센터(상탑 P.B),0,37.40549,127.14823,NaN
6,7,테크노 단일로,0,37.40486,127.15121,NaN
7,8,분당테크노파크A동,0,37.40499,127.15034,NaN
8,9,분당테크노파크C동(분당테크노파크),0,37.40545,127.15526,NaN
9,10,야탑차량등록사업소,0,37.40682,127.15470,NaN


In [3]:
# 1*. 교차로정보 수정 (데모버전)
intersections[:10]

,INT_NO,INT_NAME,SA_NO,INT_LAT,INT_LNG,IS_SMART,node_ids,X,Y,ways
0,175,한솔삼거리(한솔마을3거리),0,37.36857,127.11478,NaN,n0,437.586026,566.10,3
1,176,느티마을4단지,0,37.36724,127.11480,NaN,n1,439.350485,418.47,3
2,177,한솔6단지(정자동사무소),0,37.36619,127.11480,NaN,n2,439.350485,301.92,4
3,178,신기4거리,0,37.36346,127.11480,NaN,n3,439.350485,-1.11,4
4,201,느티마을4거리,0,37.36854,127.10984,0.0,n4,1.764460,562.77,4
5,202,느티마을 안촌유치원,0,37.36855,127.11241,NaN,n5,228.497542,563.88,2
6,206,상록마을 321동,0,37.36483,127.10981,NaN,n6,-0.882230,150.96,2
7,210,상록4거리(상록마을입구4거리),0,37.36347,127.10982,0.0,n7,0.000000,0.00,4
8,211,상록마을단일로,0,37.36346,127.11237,NaN,n8,224.968622,-1.11,2
9,212,상록마을302동 단일로,0,37.36347,127.11306,NaN,n9,285.842484,0.00,2


In [4]:
# 2. 현시정보 (phase_config, 테이블명 : S_INT_PHASE_CONFIG)
phase_config_original[:10]

,INT_NO,INT_PHASE_NO,INT_PLAN_CLSS,INT_RING,INT_FLOW_NO,INT_MIN_SPLIT,INT_MAX_SPLIT,INT_YELLOW,INT_MAIN_PHASE,INT_LANE,INT_PHASE_LAT1,INT_PHASE_LNG1,INT_PHASE_LAT2,INT_PHASE_LNG2,INT_PHASE_LAT3,INT_PHASE_LNG3
0,204,1,0,0,8,0,0,0,0,0,37.36581,127.10822,37.36614,127.10822,37.36626,127.10822
1,204,1,0,1,4,0,0,0,0,0,37.36646,127.10797,37.36614,127.10797,37.36602,127.10797
2,204,2,0,0,17,0,0,0,0,0,19.69448,117.99260,19.69448,117.99260,19.69448,117.99260
3,205,1,0,0,7,0,0,0,0,0,37.36595,127.10607,37.36563,127.10608,37.36563,127.10622
4,205,1,0,1,4,0,0,0,0,0,37.36595,127.10582,37.36563,127.10583,37.36551,127.10583
5,205,2,0,1,3,0,0,0,0,0,37.36530,127.10583,37.36563,127.10583,37.36563,127.10568
6,205,2,0,0,8,0,0,0,0,0,37.36530,127.10608,37.36563,127.10608,37.36575,127.10607
7,205,3,0,0,17,0,0,0,0,0,19.69448,117.99260,19.69448,117.99260,19.69448,117.99260
8,205,4,0,0,6,0,0,0,0,0,37.36573,127.10636,37.36573,127.10595,37.36573,127.10580
9,205,4,0,1,1,0,0,0,0,0,37.36553,127.10636,37.36553,127.10595,37.36541,127.10595


In [5]:
# 2*. 현시정보 수정 (데모버전)
phase_config[:10]

,INT_NO,ways,INT_PHASE_NO,INT_PLAN_CLSS,INT_RING,INT_FLOW_NO,INT_MIN_SPLIT,INT_MAX_SPLIT,INT_YELLOW,INT_MAIN_PHASE,INT_LANE,INT_PHASE_LAT1,INT_PHASE_LNG1,INT_PHASE_LAT2,INT_PHASE_LNG2,INT_PHASE_LAT3,INT_PHASE_LNG3
20,175,3,1,0,A,8,18,32,4,20,0,37.36825,127.11491,37.36857,127.11490,37.36869,127.11491
19,175,3,1,0,B,4,13,27,4,20,0,37.36890,127.11467,37.36857,127.11465,37.36845,127.11466
21,175,3,2,0,A,7,19,27,4,10,0,37.36889,127.11491,37.36857,127.11490,37.36856,127.11505
22,175,3,2,0,B,3,35,46,4,10,0,37.36824,127.11466,37.36857,127.11465,37.36857,127.11451
23,175,3,3,0,A,6,26,38,4,10,0,37.36867,127.11519,37.36867,127.11478,37.36867,127.11463
24,175,3,3,0,B,1,36,52,4,10,0,37.36847,127.11518,37.36847,127.11478,37.36835,127.11478
25,175,3,4,0,A,5,29,48,4,10,0,37.36866,127.11437,37.36867,127.11478,37.36878,127.11478
26,175,3,4,0,B,2,17,24,4,10,0,37.36847,127.11437,37.36847,127.11478,37.36847,127.11493
27,176,3,1,0,A,8,37,53,4,20,0,37.36691,127.11493,37.36724,127.11493,37.36735,127.11493
28,176,3,1,0,B,4,25,30,4,20,0,37.36756,127.11467,37.36723,127.11468,37.36712,127.11468


In [6]:
### tplan
tplan_path = os.path.join('daily/S_INT_TPLAN.csv')
tplan = pd.read_csv(tplan_path)
# cycleplan
cycleplan_path = os.path.join('daily/S_SA_CYCLE_PLAN.csv')
cycleplan = pd.read_csv(cycleplan_path)
# wplan
wplan_path = os.path.join('daily/S_SA_WPLAN.csv')
wplan = pd.read_csv(wplan_path)
# dplan
dplan_path = os.path.join('daily/S_SA_DPLAN.csv')
dplan = pd.read_csv(dplan_path)
# history
history_path = os.path.join('realtime/S_TOD_HIS.csv')
history= pd.read_csv(history_path)

In [7]:
# 3. 신호계획정보 (tplan, wplan, dplan, 테이블명 : S_INT_TPLAN, S_INT_WPLAN, S_INT_DPLAN)
display(tplan[:10])
display(wplan[:10])
display(dplan[:10])

,INT_NO,INT_PLAN_NO,INT_PLAN_INDEX,INT_ASPLIT1,INT_ASPLIT2,INT_ASPLIT3,INT_ASPLIT4,INT_ASPLIT5,INT_ASPLIT6,INT_ASPLIT7,INT_ASPLIT8,INT_BSPLIT1,INT_BSPLIT2,INT_BSPLIT3,INT_BSPLIT4,INT_BSPLIT5,INT_BSPLIT6,INT_BSPLIT7,INT_BSPLIT8,INT_OFFSET
0,806,1,1,60,50,30,20,NaN,NaN,NaN,NaN,60,50,30,20,NaN,NaN,NaN,NaN,NaN
1,808,1,1,60,50,30,20,NaN,NaN,NaN,NaN,60,50,30,20,NaN,NaN,NaN,NaN,NaN
2,809,1,1,60,50,30,20,NaN,NaN,NaN,NaN,60,50,30,20,NaN,NaN,NaN,NaN,NaN
3,810,1,1,60,50,30,20,NaN,NaN,NaN,NaN,60,50,30,20,NaN,NaN,NaN,NaN,NaN


,SA_NO,SA_WEEK,SA_DPLAN
0,30,1,1
1,30,2,1
2,30,3,1
3,30,4,1
4,30,5,1
5,30,6,1
6,30,7,1
7,1,2,1
8,1,3,1
9,1,1,1


,SA_NO,SA_DPLAN_NO,SA_DPLAN_INDEX,SA_START_HOUR,SA_START_MIN,SA_CYCLE_INDEX,SA_OFFSET,SA_SPLIT
0,1,1,1,0,0,1,1,1
1,1,1,2,6,0,2,2,2
2,1,1,3,10,0,3,5,5
3,1,1,4,16,0,2,2,2
4,1,1,5,0,0,0,0,0
5,1,1,6,0,0,0,0,0
6,1,1,7,0,0,0,0,0
7,1,1,8,0,0,0,0,0
8,1,1,9,0,0,0,0,0
9,1,1,10,0,0,0,0,0


In [8]:
# 4. 신호이력정보 (history, 테이블명 : S_TOD_HIS)
history

,INT_NO,INT_CREATE_DATE,INT_SPLIT1,INT_SPLIT2,INT_SPLIT3,INT_SPLIT4,INT_SPLIT5,INT_SPLIT6,INT_SPLIT7,INT_SPLIT8,INT_OFFSET,INT_CYCLE
0,743,2023-10-18 08:10:00,0,0,0,0,0,0,NaN,NaN,0,0
1,743,2023-10-18 08:13:00,0,0,0,0,0,0,NaN,NaN,0,0
2,743,2023-10-18 08:14:00,0,0,0,0,0,0,NaN,NaN,0,0
3,743,2023-10-18 08:16:00,0,0,0,0,0,0,NaN,NaN,0,0
4,743,2023-10-18 08:17:00,0,0,0,0,0,0,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
11093,477,2023-10-18 08:05:00,0,0,0,0,0,0,NaN,NaN,0,0
11094,477,2023-10-18 08:06:00,0,0,0,0,0,0,NaN,NaN,0,0
11095,477,2023-10-18 08:07:00,0,0,0,0,0,0,NaN,NaN,0,0
11096,477,2023-10-18 08:08:00,0,0,0,0,0,0,NaN,NaN,0,0
